In [12]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import dotenv

dotenv.load_dotenv()

base_path = os.getenv("BASE_PATH", ".")
API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")

embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    openai_api_key=API_KEY,
)

vectordb_path = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0529\chroma_db"
vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embedding,
)


## query 1
- 심부전으로 입원한 환자의 초기 평가 항목은?

In [39]:
query1 = [[],[]]

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 10,
    }
)
query = "심부전으로 입원한 환자의 초기 평가 항목은?"
docs = retriever.invoke(query)
print(f"\n{query}")
for doc in docs:
    source = doc.metadata.get('source', 'Unknown source')
    x = os.path.basename(source)
    print(f"  {x[13:-5].strip()}")
    query1[0].append(x[13:-5].strip())



심부전으로 입원한 환자의 초기 평가 항목은?
  순환기_11. 심부전_1. 심부전 개요_81
  응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
  외과_10. 외상_14. 전문 외상 처치술 (ATLS)_1472
  외과_5. 수술 전 처치_2. 수술 전 위험도 평가_473
  순환기_11. 심부전_4. 우심부전_79
  전공의 내과_3. 노인 의학_1. 포괄적 노인 평가_1310
  순환기_11. 심부전_2. 만성 심부전_82
  외과_2. 쇼크 _1. 쇼크_447
  순환기_2. 심혈관계 증상_3. 호흡곤란 (Dyspnea)_63
  외과_9. 심폐소생술_6. 성인 전문소생술 (ACLS) - 빈맥_1435


In [40]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
    }
)
query = "심부전으로 입원한 환자의 초기 평가 항목은?"
docs = retriever.invoke(query)
print(f"\n{query}")
for doc in docs:
    source = doc.metadata.get('source', 'Unknown source')
    x = os.path.basename(source)
    print(f"  {x[13:-5].strip()}")
    query1[1].append(x[13:-5].strip())



심부전으로 입원한 환자의 초기 평가 항목은?
  순환기_11. 심부전_1. 심부전 개요_81
  응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
  외과_5. 수술 전 처치_2. 수술 전 위험도 평가_473
  전공의 내과_3. 노인 의학_1. 포괄적 노인 평가_1310
  외과_9. 심폐소생술_6. 성인 전문소생술 (ACLS) - 빈맥_1435
  신경과_1. 신경과 총론_3. 신경계 진찰_1225
  순환기_4. 심초음파_3. 좌심실 기능 평가_1496
  내분비_5. 부신 질환_4. 부신우연종_273
  전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조치_1266
  외과_9. 심폐소생술_11. 신생아 소생술 (Neonatal resuscitation)_1433


In [51]:
import pandas as pd
df = pd.DataFrame(query1, index=["similarity", "mmr",]).T
df['일치여부'] = df['similarity'] == df['mmr']

# sim이 mmr에 존재하는 위치
def sim_in_mmr(row):
    if row['similarity'] == row['mmr']:
        return ""
    try:
        return f"{query1[1].index(row['similarity'])+1}번째에 존재"
    except ValueError:
        return "-"

# mmr이 sim에 존재하는 위치
def mmr_in_sim(row):
    if row['similarity'] == row['mmr']:
        return ""
    try:
        return f"{query1[0].index(row['mmr'])+1}번째에 존재"
    except ValueError:
        return "-"

# 열 추가
df['sim이 mmr에'] = df.apply(sim_in_mmr, axis=1)
df['mmr이 sim에'] = df.apply(mmr_in_sim, axis=1)

df


,similarity,mmr,일치여부,sim이 mmr에,mmr이 sim에
0,순환기_11. 심부전_1. 심부전 개요_81,순환기_11. 심부전_1. 심부전 개요_81,True,,
1,응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271,응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271,True,,
2,외과_10. 외상_14. 전문 외상 처치술 (ATLS)_1472,외과_5. 수술 전 처치_2. 수술 전 위험도 평가_473,False,-,4번째에 존재
3,외과_5. 수술 전 처치_2. 수술 전 위험도 평가_473,전공의 내과_3. 노인 의학_1. 포괄적 노인 평가_1310,False,3번째에 존재,6번째에 존재
4,순환기_11. 심부전_4. 우심부전_79,외과_9. 심폐소생술_6. 성인 전문소생술 (ACLS) - 빈맥_1435,False,-,10번째에 존재
5,전공의 내과_3. 노인 의학_1. 포괄적 노인 평가_1310,신경과_1. 신경과 총론_3. 신경계 진찰_1225,False,4번째에 존재,-
6,순환기_11. 심부전_2. 만성 심부전_82,순환기_4. 심초음파_3. 좌심실 기능 평가_1496,False,-,-
7,외과_2. 쇼크 _1. 쇼크_447,내분비_5. 부신 질환_4. 부신우연종_273,False,-,-
8,순환기_2. 심혈관계 증상_3. 호흡곤란 (Dyspnea)_63,전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조...,False,-,-
9,외과_9. 심폐소생술_6. 성인 전문소생술 (ACLS) - 빈맥_1435,외과_9. 심폐소생술_11. 신생아 소생술 (Neonatal resuscitatio...,False,5번째에 존재,-


## query 2.
- 심부전으로 입원한 환자에게 처음 시행해야 할 처치는?

In [32]:
query2 = [[],[]]
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 10,
    }
)
query = "심부전으로 입원한 환자에게 처음 시행해야 할 처치는?"
docs = retriever.invoke(query)
print(f"\n{query}")
for doc in docs:
    source = doc.metadata.get('source', 'Unknown source')
    x = os.path.basename(source)
    print(f"  {x[13:-5].strip()}")
    query2[0].append(x[13:-5].strip())



심부전으로 입원한 환자에게 처음 시행해야 할 처치는?
  응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
  외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperative VTE prophylaxis)_480
  부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)_1424
  외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인지 및 기본 소생술(BLS)_1430
  외과_5. 수술 전 처치_7. 수술 전 계통별 평가 5- 내분비계 평가_475
  전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조치_1266
  순환기_11. 심부전_4. 우심부전_79
  전공의 내과_1. 의료관련감염 예방_1. 손위생_1319
  외과_5. 수술 전 처치_3. 수술 전 계통별 평가 1- 심혈관계 평가_476
  외과_5. 수술 전 처치_8. 수술 전 계통별 평가 6- 혈액학적 평가_478


In [33]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,
    }
)
query = "심부전으로 입원한 환자에게 처음 시행해야 할 처치는?"
docs = retriever.invoke(query)
print(f"\n{query}")
for doc in docs:
    source = doc.metadata.get('source', 'Unknown source')
    x = os.path.basename(source)
    print(f"  {x[13:-5].strip()}")
    query2[1].append(x[13:-5].strip())



심부전으로 입원한 환자에게 처음 시행해야 할 처치는?
  응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
  외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperative VTE prophylaxis)_480
  부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)_1424
  외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인지 및 기본 소생술(BLS)_1430
  전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조치_1266
  순환기_11. 심부전_4. 우심부전_79
  전공의 내과_1. 의료관련감염 예방_1. 손위생_1319
  순환기_15. 감염 심내막염_1. 감염 심내막염_116
  외과_13. 뱀물림과 동물물림_1. 뱀 물림_508
  순환기_19. 말초혈관질환_7. 심부정맥혈전증 (DVT)_528


In [52]:
import pandas as pd
df = pd.DataFrame(query2, index=["similarity", "mmr",]).T
df['일치여부'] = df['similarity'] == df['mmr']

# sim이 mmr에 존재하는 위치
def sim_in_mmr(row):
    if row['similarity'] == row['mmr']:
        return ""
    try:
        return f"{query2[1].index(row['similarity'])+1}번째에 존재"
    except ValueError:
        return "X"

# mmr이 sim에 존재하는 위치
def mmr_in_sim(row):
    if row['similarity'] == row['mmr']:
        return ""
    try:
        return f"{query2[0].index(row['mmr'])+1}번째에 존재"
    except ValueError:
        return "X"

# 열 추가
df['sim이 mmr에'] = df.apply(sim_in_mmr, axis=1)
df['mmr이 sim에'] = df.apply(mmr_in_sim, axis=1)

df


,similarity,mmr,일치여부,sim이 mmr에,mmr이 sim에
0,응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271,응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271,True,,
1,외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperativ...,외과_5. 수술 전 처치_9. 수술 전후 심부정맥혈전 예방 (perioperativ...,True,,
2,부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)_1424,부인과_19. 자궁 경부암_2. 자궁 경부암 (Cervical cancer)_1424,True,,
3,외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인...,외과_9. 심폐소생술_2. 성인 전문소생술 (ACLS) - 심정지 1 - 심정지 인...,True,,
4,외과_5. 수술 전 처치_7. 수술 전 계통별 평가 5- 내분비계 평가_475,전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조...,False,X,6번째에 존재
5,전공의 내과_2. 의료인의 감염성 질환 노출 시 조치_2. 공기매개감염 노출 시 조...,순환기_11. 심부전_4. 우심부전_79,False,5번째에 존재,7번째에 존재
6,순환기_11. 심부전_4. 우심부전_79,전공의 내과_1. 의료관련감염 예방_1. 손위생_1319,False,6번째에 존재,8번째에 존재
7,전공의 내과_1. 의료관련감염 예방_1. 손위생_1319,순환기_15. 감염 심내막염_1. 감염 심내막염_116,False,7번째에 존재,X
8,외과_5. 수술 전 처치_3. 수술 전 계통별 평가 1- 심혈관계 평가_476,외과_13. 뱀물림과 동물물림_1. 뱀 물림_508,False,X,X
9,외과_5. 수술 전 처치_8. 수술 전 계통별 평가 6- 혈액학적 평가_478,순환기_19. 말초혈관질환_7. 심부정맥혈전증 (DVT)_528,False,X,X


## fetch K

In [67]:
query2 = [[],[]]
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k" :20,
        "lambda_mult": 0.1
    }
)
query = "급성 심부전으로 입원한 환자에게 처음 시행해야 할 처치는?"
docs = retriever.invoke(query)
print(f"\n{query}")
for doc in docs:
    source = doc.metadata.get('source', 'Unknown source')
    x = os.path.basename(source)
    print(f"  {x[13:-5].strip()}")
    query2[0].append(x[13:-5].strip())




급성 심부전으로 입원한 환자에게 처음 시행해야 할 처치는?
  응급의학_2. 외상 환자의 응급처치_1. 응급처치_1271
  순환기_11. 심부전_4. 우심부전_79
  전공의 외과_4. 영양지원_1. 영양지원_1324
  외과_9. 심폐소생술_11. 신생아 소생술 (Neonatal resuscitation)_1433
  순환기_15. 감염 심내막염_1. 감염 심내막염_116
